<a href="https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_inferencia_espa%C3%B1ol(ckpt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="ffc800"> **Cuaderno de inferencia de [Piper.](https://github.com/rhasspy/piper)**
## ![Piper logo](https://contribute.rhasspy.org/img/logo.png)

---

- Cuaderno creado por [rmcpantoja](http://github.com/rmcpantoja)
- Colaborador y traductor: [Xx_Nessu_xX](https://fakeyou.com/profile/Xx_Nessu_xX)

# <font color="ffc800">🏁 ***Primeros pasos.*** 🏁

In [ ]:
#@markdown # <font color="ffc800">**Instalar el software y ajustes.** 📦
#@markdown ---
#@markdown *El sintetizador de voz y otras dependencias importantes se instalarán en esta celda. Pero primero, algunos ajustes:*

#@markdown #### ¿Quieres activar la accesibilidad mejorada?
#@markdown *Esta funcionalidad de Accesibilidad Mejorada está diseñada para personas con discapacidad visual, en la que la mayor parte de la interfaz puede utilizarse mediante guías de voz.*
enhanced_accessibility = True #@param {type:"boolean"}
#@markdown ---

#@markdown #### Por favor, selecciona tu idioma:
lang_select = "Español" #@param ["English", "Español"]
if lang_select == "English":
    lang = "en"
elif lang_select == "Español":
    lang = "es"
else:
    raise Exception("Idioma no soportado.")
#@markdown ---

#@markdown #### <font color="orange">**¿Deseas utilizar la GPU para la inferencia? (Recomendable.)**

#@markdown *La GPU puede habilitarse en la sección <font color="orange">Entorno de ejecución</font> del cuaderno, y este paso debe realizarse antes de conectarse a este.<br>Debes entrar a: <font color="orange">"Entorno de ejecución > Cambiar tipo de entorno de ejecución > T4 GPU".</font><br>La GPU puede conducir a una mayor velocidad de respuesta en la inferencia, pero se puede utilizar la CPU, por ejemplo, si tu tiempo de ejecución en colab para utilizar las GPU se ha terminado.*
use_gpu = True #@param {type:"boolean"}

if enhanced_accessibility:
    from google.colab import output
    guideurl = f"https://github.com/rmcpantoja/piper/blob/master/notebooks/wav/{lang}"
    def playaudio(filename, extension = "wav"):
        return output.eval_js(f'new Audio("{guideurl}/{filename}.{extension}?raw=true").play()')

%cd /content
print("\033[93mInstalando.")
if enhanced_accessibility:
    playaudio("installing")
!git clone -q https://github.com/rmcpantoja/piper
%cd /content/piper/src/python
#!pip install -q -r requirements.txt
!pip install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy>=1.19.0 onnxruntime>=1.11.0 pytorch-lightning==1.7.0 torch==1.11.0
!pip install -q torchtext==0.12.0 torchvision==0.12.0
#!pip install -q torchtext==0.14.1 torchvision==0.14.1
# fixing recent compativility isswes:
!pip install -q torchaudio==0.11.0 torchmetrics==0.11.4
!bash build_monotonic_align.sh
import os
if not os.path.exists("/content/piper/src/python/lng"):
  !cp -r "/content/piper/notebooks/lng" /content/piper/src/python/lng
import sys
sys.path.append('/content/piper/notebooks')
from translator import *
lan = Translator()
print("\033[93mComprobando GPU.")
gpu_info = !nvidia-smi
if use_gpu and any('not found' in info for info in gpu_info[0].split(':')):
    if enhanced_accessibility:
        playaudio("nogpu")
    raise Exception(lan.translate(lang, "La casilla Usar GPU está marcada, pero no tienes un runtime de GPU."))
elif not use_gpu and not any('not found' in info for info in gpu_info[0].split(':')):
    if enhanced_accessibility:
        playaudio("gpuavailable")
    raise Exception(lan.translate(lang, "La casilla Usar GPU no está marcada, sin embargo estás usando un entorno de ejecución GPU. Le recomendamos que marque la casilla Usar GPU para aprovecharla."))

if enhanced_accessibility:
    playaudio("installed")
print("\033[93mCompletado.")

In [ ]:
#@markdown # <font color="ffc800">**Descargar modelo y archivo config.** 📥
#@markdown ---
%cd /content/piper/src/python
import os
#@markdown #### <font color="orange">**ID del modelo o link (formato .ckpt):**
model_url_or_id = "" #@param {type:"string"}
if model_url_or_id == "" or model_url_or_id == "http" or model_url_or_id == "1":
    if enhanced_accessibility:
        playaudio("noid")
    raise Exception(lan.translate(lang, "Invalid link or ID!"))
print("\033[93mDescargando paquete de voz.")
if enhanced_accessibility:
    playaudio("downloading")
if model_url_or_id.startswith("1"):
    !gdown -q "{model_url_or_id}"
elif model_url_or_id.startswith("https://drive.google.com/file/d/"):
    !gdown -q "{model_url_or_id}" --fuzzy
else:
    !wget -q "{model_url_or_id}"
#@markdown ---
#@markdown #### <font color="orange">**ID o URL del archivo config.json:**
config_url_or_id = "" #@param {type:"string"}
if config_url_or_id.startswith("1"):
    !gdown -q "{config_url_or_id}"
elif config_url_or_id.startswith("https://drive.google.com/file/d/"):
    !gdown -q "{config_url_or_id}" --fuzzy
else:
    !wget -q "{config_url_or_id}"
#@markdown ---
if enhanced_accessibility:
    playaudio("downloaded")
print("\033[93mPaquete de voz descargado.")

# <font color="ffc800">🔊 ***Inferencia.*** 🔊

In [ ]:
#@markdown # <font color="ffc800">**Iniciar inferencia.** 🤖
#@markdown ---
#@markdown #### Antes de que disfrutes... <font color="orange">**¡Algunas notas!**
#@markdown * <font color="orange">Puedes ejecutar la celda para descargar paquetes de voz y descargar las voces que desees en cualquier momento, ¡incluso si ejecutas esta celda antes!</font>
#@markdown * <font color="orange">Cuando descargues una voz nueva, vuelve a ejecutar esta celda y podrás alternar entre todas las que hayas descargado. Increíble, ¿verdad?

#@markdown Ahora sí, disfruta.

%cd /content/piper/src/python
# original: infer.py
import json
import logging
import sys
from pathlib import Path
from enum import Enum
from typing import Iterable, List, Optional, Union
import torch
from piper_train.vits.lightning import VitsModel
from piper_train.vits.utils import audio_float_to_int16
from piper_train.vits.wavfile import write as write_wav
import numpy as np
import glob
import ipywidgets as widgets
from IPython.display import display, Audio, Markdown, clear_output
from piper_phonemize import phonemize_codepoints, phonemize_espeak, tashkeel_run

_LOGGER = logging.getLogger("piper_train.infer_onnx")

def detect_ckpt_models(path):
    ckpt_models = glob.glob(path + '/*.ckpt')
    if len(ckpt_models) > 1:
        return ckpt_models
    elif len(ckpt_models) == 1:
        return ckpt_models[0]
    else:
        return None


def main():
    """Main entry point"""
    models_path = "/content/piper/src/python"
    logging.basicConfig(level=logging.DEBUG)
    model = None
    ckpt_models = detect_ckpt_models(models_path)
    speaker_selection = widgets.Dropdown(
        options=[],
        description=f'{lan.translate(lang, "Select speaker")}:',
        layout={'visibility': 'hidden'}
    )
    if ckpt_models is None:
        if enhanced_accessibility:
            playaudio("novoices")
        raise Exception(lan.translate(lang, "No downloaded voice packages!"))
    elif isinstance(ckpt_models, str):
        ckpt_model = ckpt_models
        model, config = load_ckpt(ckpt_model)
        if config["num_speakers"] > 1:
            speaker_selection.options = config["speaker_id_map"].values()
            speaker_selection.layout.visibility = 'visible'
            preview_sid = 0
            if enhanced_accessibility:
                playaudio("multispeaker")
        else:
            speaker_selection.layout.visibility = 'hidden'
            preview_sid = None

        if enhanced_accessibility:
            inferencing(
                model,
                config,
                preview_sid,
                lan.translate(
                    config["espeak"]["voice"][:2],
                    "Interface openned. Write your texts, configure the different synthesis options or download all the voices you want. Enjoy!"
                )
            )
    else:
        voice_model_names = []
        for current in ckpt_models:
            voice_struct = current.split("/")[5]
            voice_model_names.append(voice_struct)
        if enhanced_accessibility:
            playaudio("selectmodel")
        selection = widgets.Dropdown(
            options=voice_model_names,
            description=f'{lan.translate(lang, "Select voice package")}:',
        )
        load_btn = widgets.Button(
            description=lan.translate(lang, "Load it!")
        )
        config = None
        def load_model(button):
            nonlocal config
            global ckpt_model
            nonlocal model
            nonlocal models_path
            selected_voice = selection.value
            ckpt_model = f"{models_path}/{selected_voice}"
            model, config = load_ckpt(ckpt_model)
            if enhanced_accessibility:
                playaudio("loaded")
            if config["num_speakers"] > 1:
                speaker_selection.options = config["speaker_id_map"].values()
                speaker_selection.layout.visibility = 'visible'
                if enhanced_accessibility:
                    playaudio("multispeaker")
            else:
                speaker_selection.layout.visibility = 'hidden'

        load_btn.on_click(load_model)
        display(selection, load_btn)
    display(speaker_selection)
    speed_slider = widgets.FloatSlider(
        value=1,
        min=0.25,
        max=4,
        step=0.1,
        description=lan.translate(lang, "Rate scale"),
        orientation='horizontal',
    )
    noise_scale_slider = widgets.FloatSlider(
        value=0.667,
        min=0.25,
        max=4,
        step=0.1,
        description=lan.translate(lang, "Phoneme noise scale"),
        orientation='horizontal',
    )
    noise_scale_w_slider = widgets.FloatSlider(
        value=1,
        min=0.25,
        max=4,
        step=0.1,
        description=lan.translate(lang, "Phoneme stressing scale"),
        orientation='horizontal',
    )
    play = widgets.Checkbox(
        value=True,
        description=lan.translate(lang, "Auto-play"),
        disabled=False
    )
    text_input = widgets.Text(
        value='',
        placeholder=f'{lan.translate(lang, "Enter your text here")}:',
        description=lan.translate(lang, "Text to synthesize"),
        layout=widgets.Layout(width='80%')
    )
    synthesize_button = widgets.Button(
        description=lan.translate(lang, "Synthesize"),
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip=lan.translate(lang, "Click here to synthesize the text."),
        icon='check'
    )
    close_button = widgets.Button(
        description=lan.translate(lang, "Exit"),
        tooltip=lan.translate(lang, "Closes this GUI."),
        icon='check'
    )

    def on_synthesize_button_clicked(b):
        if model is None:
            if enhanced_accessibility:
                playaudio("nomodel")
            raise Exception(lan.translate(lang, "You have not loaded any model from the list!"))
        text = text_input.value
        if config["num_speakers"] > 1:
            sid = speaker_selection.value
        else:
            sid = None
        rate = speed_slider.value
        noise_scale = noise_scale_slider.value
        noise_scale_w = noise_scale_w_slider.value
        auto_play = play.value
        inferencing(model, config, sid, text, rate, noise_scale, noise_scale_w, auto_play)

    def on_close_button_clicked(b):
        clear_output()
        if enhanced_accessibility:
            playaudio("exit")

    synthesize_button.on_click(on_synthesize_button_clicked)
    close_button.on_click(on_close_button_clicked)
    display(text_input)
    display(speed_slider)
    display(noise_scale_slider)
    display(noise_scale_w_slider)
    display(play)
    display(synthesize_button)
    display(close_button)

def load_ckpt(model):
    _LOGGER.debug("Loading model from %s", model)
    config = load_config(model)
    model = VitsModel.load_from_checkpoint(str(model), dataset=None)
    # Inference only
    model.eval()
    with torch.no_grad():
        model.model_g.dec.remove_weight_norm()

    _LOGGER.info("Loaded model from %s", model)
    return model, config

def load_config(model):
    with open("config.json", "r") as file:
        config = json.load(file)
    return config

PAD = "_"  # padding (0)
BOS = "^"  # beginning of sentence
EOS = "$"  # end of sentence

class PhonemeType(str, Enum):
    ESPEAK = "espeak"
    TEXT = "text"

def phonemize(config, text: str) -> List[List[str]]:
    """Text to phonemes grouped by sentence."""
    if config["phoneme_type"] == PhonemeType.ESPEAK:
        if config["espeak"]["voice"] == "ar":
            # Arabic diacritization
            # https://github.com/mush42/libtashkeel/
            text = tashkeel_run(text)
        return phonemize_espeak(text, config["espeak"]["voice"])
    if config["phoneme_type"] == PhonemeType.TEXT:
        return phonemize_codepoints(text)
    raise ValueError(f"Unexpected phoneme type: {self.config.phoneme_type}")

def phonemes_to_ids(config, phonemes: List[str]) -> List[int]:
    """Phonemes to ids."""
    id_map = config["phoneme_id_map"]
    ids: List[int] = list(id_map[BOS])
    for phoneme in phonemes:
        if phoneme not in id_map:
            print("Missing phoneme from id map: %s", phoneme)
            continue
        ids.extend(id_map[phoneme])
        ids.extend(id_map[PAD])
    ids.extend(id_map[EOS])
    return ids

def inferencing(model, config, sid, line, length_scale = 1, noise_scale = 0.667, noise_scale_w = 0.8, auto_play=True):
    audios = []
    text = phonemize(config, line)
    for phonemes in text:
        phoneme_ids = phonemes_to_ids(config, phonemes)
        num_speakers = config["num_speakers"]
        if num_speakers == 1:
            speaker_id = None # for now
        else:
            speaker_id = sid
        text = torch.LongTensor(phoneme_ids).unsqueeze(0)
        text_lengths = torch.LongTensor([len(phoneme_ids)])
        scales = [
            noise_scale,
            length_scale,
            noise_scale_w
        ]
        sid = torch.LongTensor([speaker_id]) if speaker_id is not None else None
        audio = model(
            text,
            text_lengths,
            scales,
            sid=sid
        ).detach().numpy()
        audio = audio_float_to_int16(audio.squeeze())
        audios.append(audio)
    merged_audio = np.concatenate(audios)
    sample_rate = config["audio"]["sample_rate"]
    display(Markdown(f"{line}"))
    display(Audio(merged_audio, rate=sample_rate, autoplay=auto_play))

def denoise(
    audio: np.ndarray, bias_spec: np.ndarray, denoiser_strength: float
) -> np.ndarray:
    audio_spec, audio_angles = transform(audio)

    a = bias_spec.shape[-1]
    b = audio_spec.shape[-1]
    repeats = max(1, math.ceil(b / a))
    bias_spec_repeat = np.repeat(bias_spec, repeats, axis=-1)[..., :b]

    audio_spec_denoised = audio_spec - (bias_spec_repeat * denoiser_strength)
    audio_spec_denoised = np.clip(audio_spec_denoised, a_min=0.0, a_max=None)
    audio_denoised = inverse(audio_spec_denoised, audio_angles)

    return audio_denoised


def stft(x, fft_size, hopsamp):
    """Compute and return the STFT of the supplied time domain signal x.
    Args:
        x (1-dim Numpy array): A time domain signal.
        fft_size (int): FFT size. Should be a power of 2, otherwise DFT will be used.
        hopsamp (int):
    Returns:
        The STFT. The rows are the time slices and columns are the frequency bins.
    """
    window = np.hanning(fft_size)
    fft_size = int(fft_size)
    hopsamp = int(hopsamp)
    return np.array(
        [
            np.fft.rfft(window * x[i : i + fft_size])
            for i in range(0, len(x) - fft_size, hopsamp)
        ]
    )


def istft(X, fft_size, hopsamp):
    """Invert a STFT into a time domain signal.
    Args:
        X (2-dim Numpy array): Input spectrogram. The rows are the time slices and columns are the frequency bins.
        fft_size (int):
        hopsamp (int): The hop size, in samples.
    Returns:
        The inverse STFT.
    """
    fft_size = int(fft_size)
    hopsamp = int(hopsamp)
    window = np.hanning(fft_size)
    time_slices = X.shape[0]
    len_samples = int(time_slices * hopsamp + fft_size)
    x = np.zeros(len_samples)
    for n, i in enumerate(range(0, len(x) - fft_size, hopsamp)):
        x[i : i + fft_size] += window * np.real(np.fft.irfft(X[n]))
    return x


def inverse(magnitude, phase):
    recombine_magnitude_phase = np.concatenate(
        [magnitude * np.cos(phase), magnitude * np.sin(phase)], axis=1
    )

    x_org = recombine_magnitude_phase
    n_b, n_f, n_t = x_org.shape  # pylint: disable=unpacking-non-sequence
    x = np.empty([n_b, n_f // 2, n_t], dtype=np.complex64)
    x.real = x_org[:, : n_f // 2]
    x.imag = x_org[:, n_f // 2 :]
    inverse_transform = []
    for y in x:
        y_ = istft(y.T, fft_size=1024, hopsamp=256)
        inverse_transform.append(y_[None, :])

    inverse_transform = np.concatenate(inverse_transform, 0)

    return inverse_transform


def transform(input_data):
    x = input_data
    real_part = []
    imag_part = []
    for y in x:
        y_ = stft(y, fft_size=1024, hopsamp=256).T
        real_part.append(y_.real[None, :, :])  # pylint: disable=unsubscriptable-object
        imag_part.append(y_.imag[None, :, :])  # pylint: disable=unsubscriptable-object
    real_part = np.concatenate(real_part, 0)
    imag_part = np.concatenate(imag_part, 0)

    magnitude = np.sqrt(real_part**2 + imag_part**2)
    phase = np.arctan2(imag_part.data, real_part.data)

    return magnitude, phase

main()

# <font color="orange">**Te sugiero exportar este modelo para usarlo en Piper.**
Bueno, ¿ya lo has probado? ¿Qué opinas al respecto? Si ambas respuestas son aceptables, es hora de desconectar tu sesión en este cuaderno y [exportar tu modelo](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_model_exporter.ipynb) a Piper.